In [1]:
import os
import json
import cv2

In [2]:
def convert_to_coco(base_dir, subfolder, output_json, classes):
    img_dir = os.path.join(base_dir, subfolder)
    
    coco = {
        "images": [],
        "annotations": [],
        "categories": [{"id": i, "name": name} for i, name in enumerate(classes)]
    }
    
    ann_id = 1
    obj_id = 1
    
    for filename in os.listdir(img_dir):
        if filename.endswith(".png"):
            img_path = os.path.join(img_dir, filename)
            label_path = img_path.replace(".png", ".txt")
            
            img = cv2.imread(img_path)
            if img is None:
                continue
            h, w, _ = img.shape
            
            coco["images"].append({
                "id": ann_id,
                "file_name": filename,
                "width": w,
                "height": h
            })
            
            if os.path.exists(label_path):
                with open(label_path, "r") as f:
                    for line in f:
                        parts = line.split()
                        cls_id = int(parts[0])
                        x_c, y_c, bw, bh = map(float, parts[1:])
                        
                        abs_w = bw * w
                        abs_h = bh * h
                        abs_x = (x_c * w) - (abs_w / 2)
                        abs_y = (y_c * h) - (abs_h / 2)
                        
                        coco["annotations"].append({
                            "id": obj_id,
                            "image_id": ann_id,
                            "category_id": cls_id,
                            "bbox": [abs_x, abs_y, abs_w, abs_h],
                            "area": abs_w * abs_h,
                            "iscrowd": 0
                        })
                        obj_id += 1
            ann_id += 1
            
    with open(output_json, "w") as f:
        json.dump(coco, f)



In [3]:
class_list = ["door", "wall", "window"]
convert_to_coco("data", "colorful", "colorful_train.json", class_list)
convert_to_coco("data", "black_and_white", "bw_train.json", class_list)